In [1]:
from typing import List, Optional

import panel as pn
import pandas as pd
import IPython
from IPython.display import display
from bokeh.plotting import show, output_file
from bokeh.io import output_notebook
from cfa_analysis.data_retrieval import (
    get_data_from_imf,
    rename_from_abbr_to_full_name,
    get_all_metric_data,
)
from cfa_analysis.constants import CFA_FRANC_ZONE, NON_CFA_CHUNK_1, NON_CFA_CHUNK_2
from cfa_analysis.data_cleanup import (
    remove_outliers,
    remove_future_years,
    find_outliers_IQR,
)
from cfa_analysis.presentation import generate_graph, chat_gpt_analyze_results
from cfa_analysis.median_analysis import analyze_medians, generate_median_df

output_notebook() 
pn.extension()

Loading BokehJS ...

In [2]:
all_countries = get_data_from_imf(
        "https://www.imf.org/external/datamapper/api/v1/countries"
    )
countries = {v["label"]: k for k, v in all_countries["countries"].items()}
indicators_imf_dict = get_data_from_imf(
    "https://www.imf.org/external/datamapper/api/v1/indicators"
)
chunk_1_data_non_cfa = rename_from_abbr_to_full_name(
                get_all_metric_data(NON_CFA_CHUNK_1, 'NGDP_RPCH', countries),
                all_countries)  
chunk_2_data_non_cfa = rename_from_abbr_to_full_name(
    get_all_metric_data(NON_CFA_CHUNK_2, 'NGDP_RPCH', countries),
    all_countries,
)
chunk_1_data_non_cfa.update(chunk_2_data_non_cfa)
cfa_data = rename_from_abbr_to_full_name(
    get_all_metric_data(CFA_FRANC_ZONE, 'NGDP_RPCH', countries),
    all_countries,
)
non_cfa_df = pd.DataFrame.from_dict(chunk_1_data_non_cfa).T
cfa_df = pd.DataFrame.from_dict(cfa_data).T
median_cfa_df = remove_future_years(generate_median_df(cfa_df))
median_non_cfa_df = remove_future_years(generate_median_df(non_cfa_df))
merged_df = pd.merge(median_cfa_df, median_non_cfa_df, how='inner', on='year')
merged_df.rename(columns={'median_x':'median_cfa', 'median_y': 'median_non_cfa'}, inplace=True)
plot = generate_graph(merged_df, 'NGDP_RPCH', 'gdp')

# TEST HTML

In [4]:
show(plot)

# TEST HTML2